In [1]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt

In [27]:
def linear_interpolation (x, leftPoint: tuple, rightPoint: tuple):
    x1, y1 = leftPoint
    x2, y2 = rightPoint
    
    if not (x1 <= x <= x2):
        raise Exception ("check your x value in linear interpolation")
    
    return ((x2 - x)/(x2 - x1)) * y1 + ((x - x1)/(x2 - x1)) * y2

In [28]:
def bilinear_interpolation (point: tuple, left, right, top, bottom, topLeftValue, topRightValue, bottomLeftValue, bottomRightValue):
    x, y = point
    x1, x2, y1, y2 = left, right, top, bottom
    TL, TR, BL, BR = topLeftValue, topRightValue, bottomLeftValue, bottomRightValue
    
    T = linear_interpolation(x, (x1, TL), (x2, TR))
    B = linear_interpolation(x, (x1, BL), (x2, BR))
    
    return linear_interpolation(y, (y1, T), (y2, B))

In [10]:
def nearest_neighbor_interpolation_gray (image: np.array, new_height: int, new_width: int):
    #! Check input
    if (len(image.shape) != 2):
        raise Exception ("Not a grayscale image")
    
    if (not isinstance(new_height, int) or not isinstance(new_width, int)):
        raise TypeError ("Check your new height and new width")
    
    #! Get the old height and width
    old_height, old_width = image.shape
    
    #! Get the scaled factors of both height and width
    height_scaled_factor = old_height / new_height
    width_scaled_factor = old_width / new_width
    
    #! Get the new image
    new_image = np.array([[0 for _ in range (new_width)] for _ in range (new_height)], dtype = 'uint8')
    for row in range (new_height):
        scaled_row = row * height_scaled_factor
        for col in range (new_width):
            scaled_col = col * width_scaled_factor
            
            #! Get the nearest point compared with the scaled position in the old image 
            scaled_row = (max(0, math.floor(scaled_row)) 
                          if scaled_row - max(0, math.floor(scaled_row)) >= min(old_height - 1, math.ceil(scaled_row)) - scaled_row 
                          else min(old_height - 1, math.ceil(scaled_row)))
            scaled_col = (max(0, math.floor(scaled_col)) 
                          if scaled_col - max(0, math.floor(scaled_col)) >= min(old_width - 1, math.ceil(scaled_col)) - scaled_col 
                          else min(old_width - 1, math.ceil(scaled_col)))
            
            #! Get the corresponding pixel
            new_image[row, col] = image[scaled_row, scaled_col]
     
    #! Return the new scaled image       
    return new_image

In [34]:
image = cv2.imread('sunflower.jpg', 0)
new_image = nearest_neighbor_interpolation_gray(image, image.shape[0] * 2, image.shape[1] * 2)
plt.imsave('sunflower_nearest_gray.jpg', new_image, cmap = 'gray')

In [31]:
def bilinear_interpolation_gray (image: np.array, new_height: int, new_width: int):
    #! Check input
    if (len(image.shape) != 2):
        raise Exception ("Not a grayscale image")
    
    if (not isinstance(new_height, int) or not isinstance(new_width, int)):
        raise TypeError ("Check your new height and new width")
    
    #! Get the old height and width
    old_height, old_width = image.shape
    
    #! Get the scaled factors of both height and width
    height_scaled_factor = old_height / new_height
    width_scaled_factor = old_width / new_width
    
    #! Get the new image
    new_image = np.array([[0 for _ in range (new_width)] for _ in range (new_height)], dtype = 'uint8')
    for row in range (new_height):
        scaled_row = row * height_scaled_factor
        for col in range (new_width):
            scaled_col = col * width_scaled_factor
            
            #! Get 4 corners of a bilinear interpolation
            x1 = max(0, math.floor(scaled_col))
            x2 = min(old_width - 1, math.ceil(scaled_col))
            y1 = max(0, math.floor(scaled_row))
            y2 = min(old_height - 1, math.ceil(scaled_row))
            
            #! If (scaled_row, scaled_col) is a pixel of the old image -> Get the value in the old image
            if (x1 == x2 and y1 == y2):
                new_image[row, col] = image[y1, x1]
            #! If (scaled_row, scaled_col) is on a vertical line of the old image -> Vertical linear interpolation 
            elif (x1 == x2):
                new_image[row, col] = int(linear_interpolation(scaled_row, (y1, image[y1, x1]), (y2, image[y2, x1])))
            #! If (scaled_row, scaled_col) is on a horizontal line of the old image -> Horizontal linear interpolation 
            elif (y1 == y2):
                new_image[row, col] = int(linear_interpolation(scaled_col, (x1, image[y1, x1]), (x2, image[y1, x2])))
            #! Else -> Bilinear interpolation
            else:
                new_image[row, col] = int(bilinear_interpolation((scaled_col, scaled_row), x1, x2, y1, y2, 
                                                                 image[y1, x1], image[y1, x2], image[y2, x1], image[y2, x2]))
     
    #! Return the new scaled image       
    return new_image

In [35]:
image = cv2.imread('sunflower.jpg', 0)
new_image = bilinear_interpolation_gray(image, image.shape[0] * 2, image.shape[1] * 2)
plt.imsave('sunflower_bilinear_gray.jpg', new_image, cmap = 'gray')

In [24]:
def nearest_neighbor_interpolation_color (image: np.array, new_height: int, new_width: int):
    #! Check input
    if (len(image.shape) != 3):
        raise Exception ("Not a color image")
    
    if (not isinstance(new_height, int) or not isinstance(new_width, int)):
        raise TypeError ("Check your new height and new width")
    
    #! Get the old height and width
    old_height, old_width, channel = image.shape
    
    #! Get the scaled factors of both height and width
    height_scaled_factor = old_height / new_height
    width_scaled_factor = old_width / new_width
    
    #! Get the new image
    new_image = np.array([[[0 for _ in range (channel)] for _ in range (new_width)] for _ in range (new_height)], dtype = 'uint8')
    for color in range (channel):
        for row in range (new_height):
            scaled_row = row * height_scaled_factor
            for col in range (new_width):
                scaled_col = col * width_scaled_factor
                
                #! Get the nearest point compared with the scaled position in the old image 
                scaled_row = (max(0, math.floor(scaled_row)) 
                            if scaled_row - max(0, math.floor(scaled_row)) >= min(old_height - 1, math.ceil(scaled_row)) - scaled_row 
                            else min(old_height - 1, math.ceil(scaled_row)))
                scaled_col = (max(0, math.floor(scaled_col)) 
                            if scaled_col - max(0, math.floor(scaled_col)) >= min(old_width - 1, math.ceil(scaled_col)) - scaled_col 
                            else min(old_width - 1, math.ceil(scaled_col)))
                
                #! Get the corresponding pixel
                new_image[row, col, color] = image[scaled_row, scaled_col, color]
     
    #! Return the new scaled image       
    return new_image

In [36]:
image = cv2.imread('sunflower.jpg')
new_image = nearest_neighbor_interpolation_color(image, image.shape[0] * 2, image.shape[1] * 2)
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
plt.imsave('sunflower_nearest_color.jpg', new_image)

In [37]:
def bilinear_interpolation_color (image: np.array, new_height: int, new_width: int):
    #! Check input
    if (len(image.shape) != 3):
        raise Exception ("Not a color image")
    
    if (not isinstance(new_height, int) or not isinstance(new_width, int)):
        raise TypeError ("Check your new height and new width")
    
    #! Get the old height and width
    old_height, old_width, channel = image.shape
    
    #! Get the scaled factors of both height and width
    height_scaled_factor = old_height / new_height
    width_scaled_factor = old_width / new_width
    
    #! Get the new image
    new_image = np.array([[[0 for _ in range (channel)] for _ in range (new_width)] for _ in range (new_height)], dtype = 'uint8')
    for color in range (channel):
        for row in range (new_height):
            scaled_row = row * height_scaled_factor
            for col in range (new_width):
                scaled_col = col * width_scaled_factor
                
                #! Get 4 corners of a bilinear interpolation
                x1 = max(0, math.floor(scaled_col))
                x2 = min(old_width - 1, math.ceil(scaled_col))
                y1 = max(0, math.floor(scaled_row))
                y2 = min(old_height - 1, math.ceil(scaled_row))
                
                #! If (scaled_row, scaled_col) is a pixel of the old image -> Get the value in the old image
                if (x1 == x2 and y1 == y2):
                    new_image[row, col, color] = image[y1, x1, color]
                #! If (scaled_row, scaled_col) is on a vertical line of the old image -> Vertical linear interpolation 
                elif (x1 == x2):
                    new_image[row, col, color] = int(linear_interpolation(scaled_row, (y1, image[y1, x1, color]), (y2, image[y2, x1, color])))
                #! If (scaled_row, scaled_col) is on a horizontal line of the old image -> Horizontal linear interpolation 
                elif (y1 == y2):
                    new_image[row, col, color] = int(linear_interpolation(scaled_col, (x1, image[y1, x1, color]), (x2, image[y1, x2, color])))
                #! Else -> Bilinear interpolation
                else:
                    new_image[row, col, color] = int(bilinear_interpolation((scaled_col, scaled_row), x1, x2, y1, y2, 
                                                                    image[y1, x1, color], image[y1, x2, color], image[y2, x1, color], image[y2, x2, color]))
     
    #! Return the new scaled image       
    return new_image

In [38]:
image = cv2.imread('sunflower.jpg')
new_image = bilinear_interpolation_color(image, image.shape[0] * 2, image.shape[1] * 2)
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
plt.imsave('sunflower_bilinear_color.jpg', new_image)

In [39]:
def resize_image_interpolation (image: np.array, new_height: int, new_width: int, interpolation_type: str = 'nearest'):
    #! Check input
    if (len(image.shape) not in [2,3]):
        raise Exception ("Check your image!")
    
    if (not isinstance(new_height, int) or not isinstance(new_width, int)):
        raise TypeError ("Check your new height and new width")
    
    if (interpolation_type not in ['nearest', 'bilinear']):
        raise Exception (f"{interpolation_type} does not supported")
    
    #! Classify input into an appropriate situation
    if (len(image.shape) == 2 and interpolation_type == 'nearest'):
        nearest_neighbor_interpolation_gray(image, new_height, new_width)
    
    if (len(image.shape) == 2 and interpolation_type == 'bilinear'):
        bilinear_interpolation_gray(image, new_height, new_width)
        
    if (len(image.shape) == 3 and interpolation_type == 'nearest'):
        nearest_neighbor_interpolation_color(image, new_height, new_width)
        
    if (len(image.shape) == 3 and interpolation_type == 'bilinear'):
        bilinear_interpolation_color(image, new_height, new_width)